In [150]:
import pandas as pd
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder

In [151]:
train_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/train.csv')
test_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/test.csv')


# Label Onehot encoding  

le = LabelEncoder()
le.fit(train_df['author'])
print(le.classes_)

y = le.transform(train_df['author'])
y_train = np_utils.to_categorical(y)

y_train

['EAP' 'HPL' 'MWS']


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [166]:
with open('keras_input_train.pkl', 'rb') as f:
    sequences = pickle.load(f)
with open('keras_input_test.pkl', 'rb') as f:
    test_sequences = pickle.load(f)
    
x_train = sequences
x_test = test_sequences
x_test.shape

(8392, 100)

In [153]:
model = Sequential() 
model.add(Dense(units=100,input_shape=(100,),activation="relu",name="hidden"))
model.add(Dense(units=100,activation="relu",name="hidden2"))
model.add(Dense(units=100,activation="relu",name="hidden3"))
model.add(Dense(units=3,activation="softmax",name="output"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               10100     
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
hidden3 (Dense)              (None, 100)               10100     
_________________________________________________________________
output (Dense)               (None, 3)                 303       
Total params: 30,603
Trainable params: 30,603
Non-trainable params: 0
_________________________________________________________________


In [154]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [155]:
train_history = model.fit(x_train, y_train, validation_split=0.2, epochs=50, batch_size=20, verbose=1)

Train on 15663 samples, validate on 3916 samples
Epoch 1/50
15663/15663 [==============================] - 7s - loss: 0.9857 - acc: 0.5104 - val_loss: 0.9491 - val_acc: 0.5432
Epoch 2/50
15663/15663 [==============================] - 5s - loss: 0.9294 - acc: 0.5520 - val_loss: 0.9404 - val_acc: 0.5557
Epoch 3/50
15663/15663 [==============================] - 5s - loss: 0.9037 - acc: 0.5669 - val_loss: 0.9531 - val_acc: 0.5449
Epoch 4/50
15663/15663 [==============================] - 5s - loss: 0.8747 - acc: 0.5832 - val_loss: 0.9575 - val_acc: 0.5488
Epoch 5/50
15663/15663 [==============================] - 5s - loss: 0.8452 - acc: 0.6016 - val_loss: 0.9838 - val_acc: 0.5345
Epoch 6/50
15663/15663 [==============================] - 5s - loss: 0.8119 - acc: 0.6150 - val_loss: 1.0054 - val_acc: 0.5312
Epoch 7/50
15663/15663 [==============================] - 5s - loss: 0.7758 - acc: 0.6377 - val_loss: 1.0390 - val_acc: 0.5322
Epoch 8/50
15663/15663 [==============================] - 7s -

In [158]:
scores = model.evaluate(x_train, y_train)  
print()
print(scores)

19040/19579 [============================>.] - ETA: 0s
[0.84892199458235207, 0.71913785176474676]


In [168]:
prediction = model.predict(x_test)
print()
print("Show 10 prediction result :")  
print(prediction[:10])
print(prediction.shape)


Show 10 prediction result :
[[  7.81134844e-01   1.66332573e-01   5.25325499e-02]
 [  8.76182795e-01   1.34306829e-02   1.10386498e-01]
 [  9.01210085e-02   9.08059478e-01   1.81956694e-03]
 [  1.05808917e-02   9.89417732e-01   1.36821950e-06]
 [  4.41812903e-01   1.72122553e-01   3.86064529e-01]
 [  4.53834593e-01   2.30593830e-01   3.15571606e-01]
 [  5.86507261e-01   3.61998767e-01   5.14939427e-02]
 [  1.16980942e-02   1.68732717e-03   9.86614645e-01]
 [  9.86152768e-01   1.38472561e-02   1.30533139e-12]
 [  4.96377081e-01   2.52849519e-01   2.50773311e-01]]
(8392, 3)


In [169]:
# submission

submission = pd.DataFrame(prediction, columns=le.classes_)
submission["id"] = test_df["id"]
cols = submission.columns.tolist()
cols = cols[-1:]+cols[:-1]
submission = submission[cols]
submission.head()

,id,EAP,HPL,MWS
0,id02310,0.781135,0.166333,0.052533
1,id24541,0.876183,0.013431,0.110386
2,id00134,0.090121,0.908059,0.001820
3,id27757,0.010581,0.989418,0.000001
4,id04081,0.441813,0.172123,0.386065


In [170]:
submission.to_csv('submission.csv',index=False)